<a href="https://colab.research.google.com/github/nieszkodzi/Kaggle/blob/master/Landmark_Recognition_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from keras.utils import to_categorical
from keras.applications import MobileNet
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, AveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
train_data_dir = 'images/train/'
validation_data_dir = 'images/validation/'

batch_size = 16
img_height, img_width = 160, 160

In [3]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
train_generator = ImageDataGenerator(rescale=1/255, rotation_range=90, zoom_range=0.2, horizontal_flip=True)

validation_generator = ImageDataGenerator(rescale=1/255)

# Retrieve images and their classes for train and validation sets
train_flow = train_generator.flow_from_directory(directory=train_data_dir, 
                                                 batch_size=batch_size, target_size=(img_height,img_width), 
                                                 class_mode= 'binary')

validation_flow = validation_generator.flow_from_directory(
    directory=validation_data_dir, batch_size=batch_size, 
    target_size=(img_height,img_width), class_mode= 'binary')

FileNotFoundError: ignored

In [0]:
nb_train_samples = 9893
nb_validation_samples = 1969
n_classes =  10
epochs = 10

Basic CNN: Model architecture definition

We build a sequential model and add convolutional layers and max pooling layers to it. We also add dropout layers in between, dropout randomly switches off some neurons in the network which forces the data to find new paths. Therefore, this reduces overfitting. We add dense layers at the end which are used for class prediction(1–13).

Max-pooling: A technique used to reduce the dimensions of an image by taking the maximum pixel value of a grid. This also helps reduce overfitting and makes the model more generic. The example below show how 2 x 2 max pooling works

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation= 'softmax'))

model.compile(loss="binary_crossentropy",
            optimizer="adam",
            metrics=["accuracy"])

model.summary()


# Model saving callback
checkpointer = ModelCheckpoint(filepath='basic_cnn.h5py', monitor='val_acc', save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 158, 158, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 77, 77, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 36, 36, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20736)             0         
__________

In [0]:
model.fit_generator(generator=train_flow, 
                    samples_per_epoch=nb_train_samples,
                    validation_data=validation_flow,
                    nb_val_samples=nb_validation_samples,
                    callbacks=[checkpointer, early_stopping],
                    verbose=1,
                    epochs=epochs)

/home/natalia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/natalia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<keras_pre..., validation_data=<keras_pre..., callbacks=[<keras.ca..., verbose=1, epochs=10, steps_per_epoch=618, validation_steps=1969)`
  import sys


Epoch 1/10
618/618 [==============================] - 2309s 4s/step - loss: -55.8870 - acc: 0.1001 - val_loss: -55.8139 - val_acc: 0.1001
Epoch 2/10
618/618 [==============================] - 2300s 4s/step - loss: -55.8358 - acc: 0.0999 - val_loss: -55.7606 - val_acc: 0.0999
Epoch 3/10
618/618 [==============================] - 2305s 4s/step - loss: -55.8735 - acc: 0.1003 - val_loss: -55.8463 - val_acc: 0.0999
Epoch 4/10
618/618 [==============================] - 2307s 4s/step - loss: -55.8605 - acc: 0.0998 - val_loss: -55.7494 - val_acc: 0.1006
Epoch 5/10
618/618 [==============================] - 2310s 4s/step - loss: -55.8822 - acc: 0.1004 - val_loss: -55.8198 - val_acc: 0.0996
Epoch 6/10
618/618 [==============================] - 2326s 4s/step - loss: -55.8254 - acc: 0.1004 - val_loss: -55.7948 - val_acc: 0.1003
Epoch 7/10
618/618 [==============================] - 2324s 4s/step - loss: -55.9247 - acc: 0.0997 - val_loss: -55.7290 - val_acc: 0.1000
Epoch 8/10
618/618 [==============

# 